# Model DNN

Antes de ser corrido este modelo devem ser criados os dados através do script de preprocessamento

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Dropout
import tensorflow as tf

## Carregamento dos dados

X é uma matriz de tamanho (max_sequence_len,100) 

Y é o EC Number (1-4 digitos)

In [2]:
data = pd.read_pickle("../parsed_data/data.pkl")

x = data['vectors'].tolist()
x = np.array(x, dtype='float32')


y = data['ec_number'].to_numpy()

## Conversão dos labels para One Hot Encoding

In [3]:
y = y.reshape(-1, 1)
enc = OneHotEncoder()
enc.fit(y)
y = enc.transform(y).toarray()

## Train test split

Separação dos dados em conjuntos de teste e treino.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Criação do modelo

Este modelo é um modelo dnn que usa a representação da soma de vectores num unico vetor de dimensão 100.

Esta representação foi sugerida no artigo "" onde foi usado um modelo SVM no entanto nos testes do grupo uma rende neuronal apresentou melhores resultados

O modelo aqui apresentado é composto por quatro hidden layers de grande dimensão que refletem a grande complexidade do problema, existe ainda algum dropout para rsolver problemas de overfitting


- maxlen = 700
- ecn_level = 1 a 4
- representation = 'vector'

In [5]:
model = models.Sequential()
model.add(Input(shape=(100,), dtype='float32', name='main_input'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(y[0]), activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 698, 128)          84480     
_________________________________________________________________
dropout (Dropout)            (None, 698, 128)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 56)                14392     
Total params: 362,040
Trainable params: 362,040
Non-trainable params: 0
_________________________________________________________________


## Treino do modelo

Treino do modelo construido com o conjuto de treino gerado.

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=60,
                    batch_size=128,
                    validation_split=0.2)

## Validação

Validação do modelo usando o conjunto de dados de teste

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

## Detalhes do Treino do Modelo

Alguma informação sobre o treino do modelo em gráfico

In [ ]:
history_dict = history.history

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(acc_values) + 1)

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()